In [1529]:
import random

In [1530]:
class Universitas:
    total_mahasiswa = 0
    anggaran = 0

    def __init__(self, nama_kampus, prodi):
        self.nama = nama_kampus
        self.prodi = prodi

    def hitung_dosen_kampus(self):
        total = 0
        for program_studi in self.prodi:
            total += program_studi.dosen
        return total
    
    def mahasiswa_lulus(self):
        for program_studi in self.prodi:
            program_studi.angkatan.pop(0)


In [1531]:
class Prodi:
    def __init__(self, nama, ukt, kode):
        self.nama = nama
        self.kode = kode
        self.ukt = ukt
        self.angkatan = []
        self.dosen = 0

    def tambah_angkatan_prodi(self, angkatan):
        self.angkatan.append(angkatan)

    def hitung_mahasiswa_prodi(self):
        total = 0
        for tiap_angkatan in self.angkatan:
            total += tiap_angkatan.hitung_mahasiswa_angkatan()
        return total


In [1532]:
class Angkatan:
    def __init__(self, tahun):
        self.tahun = tahun
        self.mahasiswa = []
        self.kelas = []

    def tambah_mahasiswa_angkatan(self, mahasiswa):
        self.mahasiswa.append(mahasiswa)

    def hitung_mahasiswa_angkatan(self):
        return len(self.mahasiswa)

In [1533]:
class Mahasiswa:
    def __init__(self,NIM):
        self.NIM = NIM

In [1534]:
class Kelas:
    def __init__(self,nama_kelas):
        self.nama_kelas = nama_kelas
        self.mahasiswa = []
    def tambah_mahasiswa_kelas(self,mahasiswa):
        self.mahasiswa.append(mahasiswa)
    def hitung_mahasiswa_kelas(self):
        return len(self.mahasiswa)

In [1535]:
prodi = [Prodi("Informatika",7000000,"IF"),
         Prodi("Sistem Informasi",8000000,"SI"),
         Prodi("Teknologi Indormasi",8000000,"TI"),
         Prodi("Rekayasa Perangkat Lunak",8500000,"RPL"),
         Prodi("Sains Data",7000000,"DS")]

In [1536]:
TelkomMadura = Universitas("Telkom Madura",prodi)

In [1537]:
def proses_mendaftar(NIM,tahun_angkatan):
    # MEMILIH PRODI
    prodi_terpilih = random.randint(0,4)
    mahasiswa_baru = Mahasiswa(NIM)
    prodi[prodi_terpilih].angkatan[tahun_angkatan].mahasiswa.append(mahasiswa_baru)
    
    # PEMBAYARAN
    BiayaUP3 = 8000000
    BiayaSDP = 12150000
    BiayaUKT = prodi[prodi_terpilih].ukt
    Total = BiayaSDP + BiayaUKT + BiayaUP3
    return Total

In [1538]:
def membuat_kelas(angkatan):
    for program_studi in prodi:
        total_mahasiswa = program_studi.angkatan[angkatan].hitung_mahasiswa_angkatan()
        total_kelas = total_mahasiswa // 30
        kode_kelas = 0
        for _ in range(total_kelas):
            program_studi.angkatan[angkatan].kelas.append(Kelas(f"{program_studi.kode}-{angkatan}-{kode_kelas + 1}")) # menambahkan kelas baru
            for mahasiswa in program_studi.angkatan[angkatan].mahasiswa: # looping ke mahasiswa di angkatan tertentu
                program_studi.angkatan[angkatan].kelas[kode_kelas].mahasiswa.append(mahasiswa) # menambahkan mahasiswa ke kelas
                if program_studi.angkatan[angkatan].kelas[kode_kelas].hitung_mahasiswa_kelas() == 30: # jika mahasiswa per kelas sudah 30 maka break dan buat kelas baru
                    break
            kode_kelas += 1
            
        if total_mahasiswa - total_mahasiswa * 30 != 0:
            sisa_mahasiswa = total_mahasiswa - total_mahasiswa * 30 
            for sisa in range(total_mahasiswa-sisa_mahasiswa,total_mahasiswa): # looping dari mahasiswa yang belum dapat kelas
                program_studi.angkatan[angkatan].kelas.append(Kelas(f"{program_studi.kode}-{angkatan}-{kode_kelas + 1}")) # menambahkan kelas baru
                program_studi.angkatan[angkatan].kelas[kode_kelas].mahasiswa.append(program_studi.angkatan[angkatan].mahasiswa[sisa]) # menambahkan mahasiswa sisa ke kelas baru
                if program_studi.angkatan[angkatan].kelas[kode_kelas].hitung_mahasiswa_kelas() == 30: # jika mahasiswa per kelas sudah 30 maka break dan buat kelas baru
                    break
            kode_kelas += 1
        # KEBUTUHAN DOSEN
        hitung_kebutuhan_dosen(program_studi)

In [1539]:
def hitung_kebutuhan_dosen(program_studi):
    total_mahasiswa = program_studi.hitung_mahasiswa_prodi()
    kebutuhan_dosen = total_mahasiswa // 60
    program_studi.dosen = kebutuhan_dosen

In [1540]:
def pengeluaran_kampus(bulan):
    gajiDosen = 7000000 * TelkomMadura.hitung_dosen_kampus()
    pengeluaran = 0.7 * TelkomMadura.anggaran / 12  + gajiDosen
    if bulan == 12:
        pengeluaran += 200000000
    return pengeluaran

In [1541]:
def hitung_mahasiswa(tahun_angkatan):
    total = 0
    for program_studi in prodi:
        total += program_studi.angkatan[tahun_angkatan].hitung_mahasiswa_angkatan()
    return total

In [1542]:
def bayar_ukt(self):
    pemasukan = 0
    for program_studi in prodi:
        pemasukan += program_studi.ukt * program_studi.hitung_mahasiswa_prodi()
    return pemasukan

In [1543]:
def simulasi_kampus(tahun_start,tahun_terakhir):
    angkatan = tahun_start
    growth_factor = 1.02 # kenaikan atau penurunan mahasiswa calon pendaftar
    mahasiswa = random.randint(1,100) # kemungkinan mahasiswa calon pendaftar
    nim = 1 # NIM atau ID mahasiswa
    cashflow = []
    # SIMULASI
    for tahun in range(tahun_terakhir - tahun_start):
        
        # Membuat Angkatan Baru di Tiap Prodi
        for program_studi in prodi:
            program_studi.tambah_angkatan_prodi(Angkatan(tahun))
        for bulan in range(1,13):
            pemasukan = 0
            if(bulan == 2 or bulan == 9):
                pemasukan += bayar_ukt(prodi)
            # PENDAFTARAN   
            for _ in range(30): 
                if random.random() < 0.5: # Kemungkinan kenaikan calon pendaftar
                    mahasiswa +=  int(mahasiswa * random.uniform(0.05, 0.5) * growth_factor)
                else:
                    mahasiswa -= int(mahasiswa * random.uniform(0.05, 0.5) * growth_factor)
                # Mahasiswa Mendaftar
                for _ in range(mahasiswa):
                    if random.random() < 0.5:
                        pemasukan += proses_mendaftar(nim,tahun)
                        nim += 1
                    if hitung_mahasiswa(tahun) == 600:
                        break
            
            # PEMASUKAN TIAP BULAN
            TelkomMadura.anggaran += pemasukan
        
            # PENGELUARAN TIAP BULAN
            pengeluaran = pengeluaran_kampus(bulan)
            TelkomMadura.anggaran -= pengeluaran
            cashflow.append([angkatan,bulan,pemasukan,pengeluaran,TelkomMadura.anggaran])
        
        if tahun % 4 == 0 and tahun != 0:
            TelkomMadura.mahasiswa_lulus(tahun - 4)
        
        # PEMBAGIAN KELAS
        membuat_kelas(tahun)
        angkatan += 1
        
    return cashflow

In [1544]:
result = simulasi_kampus(2021,2050)
result

Informatika (0) :
IF-0-1 = [14, 15, 20, 23, 28, 29, 34, 53, 54, 58, 64, 76, 80, 84, 85, 87, 94, 106, 113, 119, 120, 132, 133, 136, 140, 148, 151, 152, 153, 161]
Informatika (0) :
IF-0-2 = [14, 15, 20, 23, 28, 29, 34, 53, 54, 58, 64, 76, 80, 84, 85, 87, 94, 106, 113, 119, 120, 132, 133, 136, 140, 148, 151, 152, 153, 161]
Informatika (0) :
IF-0-3 = [14, 15, 20, 23, 28, 29, 34, 53, 54, 58, 64, 76, 80, 84, 85, 87, 94, 106, 113, 119, 120, 132, 133, 136, 140, 148, 151, 152, 153, 161]
Informatika (1) :
IF-1-1 = [479, 481, 482, 489, 491, 493, 498, 504, 507, 509, 511, 512, 521, 524, 529, 530, 548, 560, 569, 570, 576, 578, 584, 587, 591, 592, 595, 600, 601, 608]
Informatika (2) :
IF-2-1 = [672, 674, 680, 681, 696, 697, 706, 716, 718, 723, 733, 734, 735, 743, 752, 755, 756, 758, 764, 766, 767, 777, 780, 781, 787, 789, 791, 804, 805, 806]
Informatika (3) :
IF-3-1 = [861, 865, 867, 880, 885, 893, 904, 908, 909, 918, 926, 942, 943, 947, 951, 952, 953, 954, 960, 964, 968, 986, 989, 993, 994, 996, 100

[[2021, 1, 4572100000, 266705833.33333334, 4305394166.666667],
 [2021, 2, 1957750000, 365350076.3888889, 5897794090.277778],
 [2021, 3, 884800000, 395651321.9328703, 6386942768.344908],
 [2021, 4, 866150000, 423097078.15345293, 6829995690.191455],
 [2021, 5, 1151650000, 465595998.5945015, 7516049691.596953],
 [2021, 6, 887800000, 490224565.3431556, 7913625126.253798],
 [2021, 7, 857650000, 511657715.6981382, 8259617410.555659],
 [2021, 8, 975250000, 538700598.9490801, 8696166811.60658],
 [2021, 9, 3906800000, 735173064.0103837, 11867793747.596197],
 [2021, 10, 667600000, 731231301.9431114, 11804162445.653086],
 [2021, 11, 335300000, 708135309.3297633, 11431327136.323322],
 [2021, 12, 500700000, 896034916.2855271, 11035992220.037796],
 [2022, 1, 473050000, 706360796.1688714, 10802681423.868925],
 [2022, 2, 4341350000, 918401833.0590205, 14225629590.809904],
 [2022, 3, 362450000, 885971309.463911, 13702108281.345993],
 [2022, 4, 558500000, 866868816.4118495, 13393739464.934143],
 [2022, 

In [1545]:

# PEMBAGIAN KELAS

